In [27]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ModuleNotFoundError as e:
    print("not in colab")
    pass
import os
base_dir = "/content/drive/MyDrive/semeval2022"
if not os.path.exists(base_dir):
  !pip install -r requirements.txt
  base_dir = ""
else:
  !pip install -r /content/drive/MyDrive/semeval2022/requirements.txt
  !cp -rf /content/drive/MyDrive/semeval2022/*.py . 
  !cp -rf /content/drive/MyDrive/semeval2022/utils .
  !cp -rf /content/drive/MyDrive/semeval2022/model .

not in colab


In [1]:
from utils.util import parse_args, get_reader, load_model, get_trainer, get_out_filename, write_eval_performance, get_tagset
import time
from torch.utils.data import DataLoader
from utils.util import wnut_iob
import os

In [2]:
checkpoint = os.path.join(base_dir, "roberta-train/lightning_logs/version_0/checkpoints/epoch=4-step=4784.ckpt")
checkpoint = os.path.join(base_dir, "roberta-train/lightning_logs/version_0/checkpoints/roberta-train_timestamp_1638429490.282762_final.ckpt")
#checkpoint = "./roberta-finetune/lightning_logs/version_0/checkpoints/"
checkpoint = os.path.join(base_dir, "roberta-finetune/lightning_logs/version_1/checkpoints/finetune_timestamp_1638516609.385282_final.ckpt")
train_file = os.path.join(base_dir, "training_data/EN-English/en_train.conll")
dev_file = os.path.join(base_dir, "training_data/EN-English/en_dev.conll")
output_dir = os.path.join(base_dir, "roberta-evaluate")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
iob_tagging = wnut_iob

In [36]:
test_data = get_reader(file_path=dev_file, target_vocab=wnut_iob, max_instances=-1, max_length=55, encoder_model="roberta-base")
model = load_model(checkpoint, tag_to_id=wnut_iob)
trainer = get_trainer(is_test=True)
out = trainer.test(model, test_dataloaders=DataLoader(test_data, batch_size=16, collate_fn=model.collate_batch))
eval_file, eval_detail_file = get_out_filename(output_dir, checkpoint, prefix="eval")
write_eval_performance(out, eval_file)

2021-12-07 14:58:26 - INFO - reader - Reading file ./training_data/EN-English/en_dev.conll
2021-12-07 14:58:26 - INFO - reader - Finished reading 800 instances from file ./training_data/EN-English/en_dev.conll
Some weights of the model checkpoint at ./roberta-retrained/checkpoint-5000 were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./roberta-r

Testing: 0it [00:00, ?it/s]

/Users/malong/opt/anaconda3/envs/sem_eval/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:57: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  batch_size = bs


Testing: 100%|██████████| 100/100 [01:02<00:00,  1.54it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ALLPRED': 1241.0,
 'ALLRECALLED': 1042.0,
 'ALLTRUE': 1230.0,
 'F1@CORP': 0.7900552749633789,
 'F1@CW': 0.6277777552604675,
 'F1@GRP': 0.7823834419250488,
 'F1@LOC': 0.8506224155426025,
 'F1@PER': 0.9103214740753174,
 'F1@PROD': 0.6206896305084229,
 'MD@F1': 0.843383252620697,
 'MD@P': 0.8396454453468323,
 'MD@R': 0.8471544981002808,
 'P@CORP': 0.8461538553237915,
 'P@CW': 0.614130437374115,
 'P@GRP': 0.7704081535339355,
 'P@LOC': 0.8266128897666931,
 'P@PER': 0.8936877250671387,
 'P@PROD': 0.6293706297874451,
 'R@CORP': 0.7409326434135437,
 'R@CW': 0.6420454382896423,
 'R@GRP': 0.7947368621826172,
 'R@LOC': 0.8760683536529541,
 'R@PER': 0.9275861978530884,
 'R@PROD': 0.6122449040412903,
 'loss': 5.672953128814697,
 'micro@F1': 0.7859166264533997,
 'micro@P': 0.7824335098266602,
 'micro@R': 0.7894309163093567}
-----------

2021-12-07 14:59:31 - INFO - util - Finished writing evaluation performance for roberta-evaluate/eval_base_finetune_timestamp_1638516609.385282_final.tsv


In [35]:
batch_size = 8
test_dataloader = DataLoader(test_data, batch_size=batch_size, collate_fn=model.collate_batch)
sentences = test_data.sentences
ner_tags = test_data.ner_tags
pos_to_singel_word_map = test_data.pos_to_single_word_maps
f = open(eval_detail_file, "w")
f.write("token\tlabel\tpred\n")
f2 = open("en.pred.conll", "w")
for idx, batch in enumerate(test_dataloader):
    output = model.perform_forward_step(batch)
    pred_result = output["pred_results"]
    raw_pred_results = output["raw_pred_results"]
    for i in range(batch_size):
        sentence = sentences[idx*batch_size+i]
        pos_to_singel_word = pos_to_singel_word_map[idx*batch_size+i]
        ner_tag = ner_tags[idx*batch_size+i]
        input_ids = batch[0][i]
        pred_token_tag = pred_result[i]
        raw_pred_token_tag = raw_pred_results[i]
        metadata_token_tag = batch[3][i]
        meta_labels = []
        pred_labels = []
        sentence_subtokens = []
        for (start_pos, end_pos), (pred_start_pos, pred_end_pos) in zip(metadata_token_tag, pred_token_tag):
            sub_tokens = test_data.tokenizer.convert_ids_to_tokens(input_ids[start_pos: end_pos+1])
            sentence_subtokens.extend(sub_tokens)
            pred_sub_tokens = test_data.tokenizer.convert_ids_to_tokens(input_ids[start_pos: end_pos+1])
            tag = metadata_token_tag[(start_pos, end_pos)]
            pred_tag = pred_token_tag[(pred_start_pos, pred_end_pos)]
            for sub_token1, sub_token2 in zip(sub_tokens, pred_sub_tokens):
                meta_labels.append(tag)
                pred_labels.append(pred_tag)
                #f.write("{}{}{}{}{}{}{}".format(sub_token1, ",", tag, ",", sub_token2, ",", pred_tag))
                #f.write("\n")
        for (start, end) in pos_to_singel_word:
            single_word_tokens = sentence_subtokens[start:end]
            word = "".join(single_word_tokens)
            if word == "almada":
                print(sentence_subtokens)
                print(pos_to_singel_word)
            word_meta_tag = ner_tag[start]
            word_pred_tag = raw_pred_token_tag[start]
            f.write("{}\t{}\t{}".format(word, word_meta_tag, word_pred_tag))
            f2.write("{}\n".format(word_pred_tag))
            f.write("\n")
        f2.write("\n")
            
f.close() 
f2.close()

['<pad>', 'it', 'was', 'con', 'structed', 'in', 'the', 'par', 'ish', 'of', 'pr', 'ag', 'al', ',', 'which', 'was', 'mer', 'ged', 'with', 'the', 'par', 'ishes', 'of', 'alm', 'ada', ',', 'c', 'ova', 'da', 'p', 'ied', 'ade', ',', 'pr', 'ag', 'al', 'e', 'c', 'ac', 'il', 'has', 'in', '2013', ',', 'into', 'the', 'mun', 'icip', 'ality', 'of', 'alm', 'ada', '<pad>']
{(1, 2): 'it', (2, 3): 'was', (3, 5): 'constructed', (5, 6): 'in', (6, 7): 'the', (7, 9): 'parish', (9, 10): 'of', (10, 13): 'pragal', (13, 14): ',', (14, 15): 'which', (15, 16): 'was', (16, 18): 'merged', (18, 19): 'with', (19, 20): 'the', (20, 22): 'parishes', (22, 23): 'of', (23, 26): 'almada,', (26, 28): 'cova', (28, 29): 'da', (29, 33): 'piedade,', (33, 36): 'pragal', (36, 37): 'e', (37, 41): 'cacilhas', (41, 42): 'in', (42, 43): '2013', (43, 44): ',', (44, 45): 'into', (45, 46): 'the', (46, 49): 'municipality', (49, 50): 'of', (50, 52): 'almada'}


In [24]:
import pandas as pd
result_data = pd.read_csv(eval_detail_file, delimiter="\t")

In [29]:
error_data = result_data[result_data["label"] != result_data["pred"]]

print(len(error_data))

630


In [37]:
test_data.tokenizer("among others , he was chairman of the standing conference of regional sports federations and member of the südwestrundfunk broadcasting council")

{'input_ids': [0, 31636, 643, 2156, 37, 21, 2243, 9, 5, 2934, 1019, 9, 2174, 1612, 32584, 1635, 8, 919, 9, 5, 579, 2768, 417, 10823, 2070, 1187, 506, 6435, 16687, 1676, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [35]:
test_data.sentences[0].split(" ")

['',
 'ease',
 'on',
 'down',
 'the',
 'road',
 '—',
 'charlie',
 'smalls',
 '(',
 'diana',
 'ross',
 'and',
 'michael',
 'jackson',
 'in',
 'the',
 'wiz',
 ')']

In [30]:
result_data

,token,label,pred
0,ease,B-CW,B-CW
1,on,I-CW,I-CW
2,down,I-CW,I-CW
3,the,I-CW,I-CW
4,road,I-CW,I-CW
...,...,...,...
13448,and,O,O
13449,other,O,O
13450,variety,O,O
13451,performers,O,O
